# 1 ✘

搜索回溯

In [6]:
from typing import List

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 0.哨
        board = [['', *b, ''] for b in board]
        board.insert(0, [''] * len(board[0]))
        board.append([''] * len(board[0]))
        # print(board)
        # 1.先将开头字母的索引记录下来
        start_letter = {}
        for x in words:
            start_letter[x[0]] = []
        # print(start_letter)
        for r in range(len(board)):
            for c in range(len(board[0])):
                char = board[r][c]
                if char in start_letter:
                    start_letter[char].append([r, c])
        # print(start_letter)
        # 2.搜索回溯
        def bfs(board, start_pos, word):
            direction = [(0, -1), (0, 1), (-1, 0), (1, 0)]
            vis = [[False] * len(board[0]) for _ in range(len(board))]
            queue = [-1, start_pos]
            word = word[1:] # 跳过第一个
            for i, char in enumerate(word):
                if isinstance(queue[0], int):
                    queue.pop(0)
                while queue and isinstance(queue[0], list):
                    r, c = queue.pop(0)
                    # print(f'({r},{c}){board[r][c]}->{char}', end='   ')
                    vis[r][c] = True
                    is_match = False                        # 上下左右能否匹配
                    queue.append(-1)    # 哨
                    for x, y in direction:
                        if board[r+x][c+y] == char and not vis[r+x][c+y]:   # 匹配字符，且未访问
                            queue.append([r+x, c+y])
                            is_match = True
                    # print(queue, is_match)
                    if not is_match:
                        queue.pop()
                        vis[r][c] = False   # 如果不能向后匹配，哪么回滚该访问记录，避免影响后续匹配
                        if queue == []:
                            return False
                    if is_match and i == len(word)-1:    # 到倒数第2个字符，并且还能上下左右还能匹配成功
                        return True

        ans = []
        for w in words:
            if start_letter[w[0]] == []:
                continue
            if len(w) == 1:
                ans.append(w)
                continue
            for start_pos in start_letter[w[0]]:
                if bfs(board, start_pos, w):
                    ans.append(w)
                    break           # 如果在一个地方匹配成功，就不再继续
                # print('---')
            # print('===')
        return ans



if __name__ == '__main__':
    board_words = [
        [
            [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
            ["oath","pea","eat","rain"],
        ],
        [
            [["a","b"],["c","d"]],
            ["abcb"],
        ],
        [
            [["a"]],
            ["a"],
        ],
        [
            [["a", "b"], ["c", "d"]],
            ["ac"],
        ],
        [
            [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]],
            ["oa","oaa"],
        ],
        [
            [["a","b","c"],["a","e","d"],["a","f","g"]],
            ["eaabcdgfa",]
        ],
        [
            [["a"]],
            ["ab"]
        ],
        [
            [["a","b"],["a","a"]],
            ["bab", "aaba"]
        ],
    ]

    for board, words in board_words:
        print(Solution().findWords(board, words))

['oath', 'eat']
[]
['a']
['ac']
['oa', 'oaa']
['eaabcdgfa']
[]
[]


# 2

广度优先有问题，会影响兄弟分支的判断。\
改为深度优先。

为了能正确回滚到分叉点，使用for循环代替栈stack。

In [2]:
from typing import List
# from pprint import pprint

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 0.哨
        board = [['', *b, ''] for b in board]
        board.insert(0, [''] * len(board[0]))
        board.append([''] * len(board[0]))
        # print(board)
        # 1.先将开头字母的索引记录下来
        start_letter = {}
        for x in words:
            start_letter[x[0]] = []
        # print(start_letter)
        for r in range(len(board)):
            for c in range(len(board[0])):
                char = board[r][c]
                if char in start_letter:
                    start_letter[char].append([r, c])
        # print(start_letter)
        # 2.搜索回溯
        def dfs(board, word, match, vis):
            # print('match:', match)
            # pprint(vis)
            # print('####\n\n')
            i, (r, c) = match

            if i == len(word)-1:
                return True

            stacks = []
            vis[r][c] = True
            direction = [(0, -1), (0, 1), (-1, 0), (1, 0)]
            for x, y in direction:
                if board[r+x][c+y] == word[i+1] and not vis[r+x][c+y]:   # 匹配字符，且未访问
                    stacks.append([i+1, (r+x, c+y)])
            result = []
            for i, (r, c) in stacks:                     # 用for循环代替stack，才能回滚访问记录 到分叉点
                result += [dfs(board, word, [i, (r, c)], vis)]
                vis[r][c] = False
            return any(result)

        ans = []
        for word in words:
            if start_letter[word[0]] == []:
                continue
            if len(word) == 1:
                ans.append(word)
                continue
            for start_pos in start_letter[word[0]]:
                match = [0, start_pos]  # board的起始坐标，word的匹配字符索引
                vis = [[False] * len(board[0]) for _ in range(len(board))]
                if dfs(board, word, match, vis):
                    ans.append(word)
                    break           # 如果在一个地方匹配成功，就不再继续
            # print('----')
        return ans



if __name__ == '__main__':
    board_words = [
        [
            [["o","a","t","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
            ["oath","pea","eat","rain"],
        ],
        [
            [["a","b"],["c","d"]],
            ["abcb"],
        ],
        [
            [["a"]],
            ["a"],
        ],
        [
            [["a", "b"], ["c", "d"]],
            ["ac"],
        ],
        [
            [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]],
            ["oa","oaa"],
        ],
        [
            [["a","f","c"],["a","e","d"],["a","f","g"]],
            ["eaafcdgfa", "eaf"]
        ],
        [
            [["a"]],
            ["ab"]
        ],
        [
            [["a","b"],["a","a"]],
            ["bab", "aaba"]
        ],
        [
            [["a","b","e"],["b","c","d"]],
            ["abcdeb"]
        ],
    ]

    for board, words in board_words:
        print(Solution().findWords(board, words))

['oath', 'eat']
[]
['a']
['ac']
['oa', 'oaa']
['eaafcdgfa']
[]
['aaba']
['abcdeb']


Time Limit Exceeded\
42/43 cases passed (N/A)

# 3

好家伙，最后一个测试用例超时，\
一看，这不是来消遣人吗，\
好吧好吧，给你再整个cache缓存已经找到的部分。（没意思）

In [21]:
# # 在dfs里写缓存还挺麻烦，lru_cache也不能用，自己写一个缓存吧。
# # 缓存也还有个问题，如果是几个分支，应该记录多个。
#
# from typing import List
# # from functools import lru_cache   # 由于不是所有参数都能hash，所以自己封装一个
# from functools import wraps
# # from collections import defaultdict
# # from pprint import pprint

# def my_cache(func):
#     cache = {}
#     @wraps(func)
#     def warp_func(*args, **kwargs):
#         word = args[1][:args[2][0]+1]
#         if word in cache:
#             return cache[word]
#         else:
#             result = func(*args, **kwargs)
#             if result:
#                 cache[word] = result
#             # print(cache)
#             return result
#     return warp_func

自己写还有一堆问题，\
换个思路，其实并不是所有参数都需要传递，\
将board和vis作为全局变量使用就行，match改造为tuple类型就可以hash了。

这么想，其实都不需要自己再实现cache了，继续使用lru_cache。

In [7]:
from typing import List
from functools import lru_cache
# from pprint import pprint


class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 0.哨
        board = [['', *b, ''] for b in board]
        board.insert(0, [''] * len(board[0]))
        board.append([''] * len(board[0]))
        # print(board)
        # 1.先将开头字母的索引记录下来
        start_letter = {}
        for x in words:
            start_letter[x[0]] = []
        # print(start_letter)
        for r in range(len(board)):
            for c in range(len(board[0])):
                char = board[r][c]
                if char in start_letter:
                    start_letter[char].append([r, c])
        # print(start_letter)
        # 2.搜索回溯
        @lru_cache()
        def dfs(word, match):                                       # 仅保留可以被hash的参数
            print('word:', word, 'match:', match)
            # pprint(vis)
            # print('####\n\n')
            i, (r, c) = match

            if i == len(word)-1:
                return True

            stacks = []
            vis[r][c] = True
            direction = [(0, -1), (0, 1), (-1, 0), (1, 0)]
            for x, y in direction:
                if board[r+x][c+y] == word[i+1] and not vis[r+x][c+y]:   # 匹配字符，且未访问
                    stacks.append((i+1, (r+x, c+y)))
            result = []
            for i, (r, c) in stacks:                     # 用for循环代替stack，才能回滚访问记录 到分叉点
                result += [dfs(word, (i, (r, c)))]
                vis[r][c] = False
            return any(result)

        # 3.循环每个词
        ans = []
        for word in words:
            if start_letter[word[0]] == []:
                continue
            if len(word) == 1:
                ans.append(word)
                continue
            for start_pos in start_letter[word[0]]:
                match = (0, tuple(start_pos))  # board的起始坐标，word的匹配字符索引
                vis = [[False] * len(board[0]) for _ in range(len(board))]
                if dfs(word, match):
                    ans.append(word)
                    break           # 如果在一个地方匹配成功，就不再继续
            # print('----')
        return ans



if __name__ == '__main__':
    board_words = [
        # [
        #     [["o","a","t","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
        #     ["oath","pea","eat","rain"],
        # ],
        # [
        #     [["a","b"],["c","d"]],
        #     ["abcb"],
        # ],
        # [
        #     [["a"]],
        #     ["a"],
        # ],
        # [
        #     [["a", "b"], ["c", "d"]],
        #     ["ac"],
        # ],
        # [
        #     [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]],
        #     ["oa","oaa"],
        # ],
        # [
        #     [["a","f","c"],["a","e","d"],["a","f","g"]],
        #     ["eaafcdgfa", "eaf"]
        # ],
        # [
        #     [["a"]],
        #     ["ab"]
        # ],
        # [
        #     [["a","b"],["a","a"]],
        #     ["bab", "aaba"]
        # ],
        # [
        #     [["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"]],
        #     ["ababababaa","ababababab","ababababac","ababababad","ababababae","ababababaf","ababababag","ababababah","ababababai","ababababaj","ababababak","ababababal","ababababam","ababababan","ababababao","ababababap","ababababaq","ababababar","ababababas","ababababat","ababababau","ababababav","ababababaw","ababababax","ababababay","ababababaz","ababababba","ababababbb","ababababbc","ababababbd","ababababbe","ababababbf","ababababbg","ababababbh","ababababbi","ababababbj","ababababbk","ababababbl","ababababbm","ababababbn","ababababbo","ababababbp","ababababbq","ababababbr","ababababbs","ababababbt","ababababbu","ababababbv","ababababbw","ababababbx","ababababby","ababababbz","ababababca","ababababcb","ababababcc","ababababcd","ababababce","ababababcf","ababababcg","ababababch","ababababci","ababababcj","ababababck","ababababcl","ababababcm","ababababcn","ababababco","ababababcp","ababababcq","ababababcr","ababababcs","ababababct","ababababcu","ababababcv","ababababcw","ababababcx","ababababcy","ababababcz","ababababda","ababababdb","ababababdc","ababababdd","ababababde","ababababdf","ababababdg","ababababdh","ababababdi","ababababdj","ababababdk","ababababdl","ababababdm","ababababdn","ababababdo","ababababdp","ababababdq","ababababdr","ababababds","ababababdt","ababababdu","ababababdv","ababababdw","ababababdx","ababababdy","ababababdz","ababababea","ababababeb","ababababec","ababababed","ababababee","ababababef","ababababeg","ababababeh","ababababei","ababababej","ababababek","ababababel","ababababem","ababababen","ababababeo","ababababep","ababababeq","ababababer","ababababes","ababababet","ababababeu","ababababev","ababababew","ababababex","ababababey","ababababez","ababababfa","ababababfb","ababababfc","ababababfd","ababababfe","ababababff","ababababfg","ababababfh","ababababfi","ababababfj","ababababfk","ababababfl","ababababfm","ababababfn","ababababfo","ababababfp","ababababfq","ababababfr","ababababfs","ababababft","ababababfu","ababababfv","ababababfw","ababababfx","ababababfy","ababababfz","ababababga","ababababgb","ababababgc","ababababgd","ababababge","ababababgf","ababababgg","ababababgh","ababababgi","ababababgj","ababababgk","ababababgl","ababababgm","ababababgn","ababababgo","ababababgp","ababababgq","ababababgr","ababababgs","ababababgt","ababababgu","ababababgv","ababababgw","ababababgx","ababababgy","ababababgz","ababababha","ababababhb","ababababhc","ababababhd","ababababhe","ababababhf","ababababhg","ababababhh","ababababhi","ababababhj","ababababhk","ababababhl","ababababhm","ababababhn","ababababho","ababababhp","ababababhq","ababababhr","ababababhs","ababababht","ababababhu","ababababhv","ababababhw","ababababhx","ababababhy","ababababhz","ababababia","ababababib","ababababic","ababababid","ababababie","ababababif","ababababig","ababababih","ababababii","ababababij","ababababik","ababababil","ababababim","ababababin","ababababio","ababababip","ababababiq","ababababir","ababababis","ababababit","ababababiu","ababababiv","ababababiw","ababababix","ababababiy","ababababiz","ababababja","ababababjb","ababababjc","ababababjd","ababababje","ababababjf","ababababjg","ababababjh","ababababji","ababababjj","ababababjk","ababababjl","ababababjm","ababababjn","ababababjo","ababababjp","ababababjq","ababababjr","ababababjs","ababababjt","ababababju","ababababjv","ababababjw","ababababjx","ababababjy","ababababjz","ababababka","ababababkb","ababababkc","ababababkd","ababababke","ababababkf","ababababkg","ababababkh","ababababki","ababababkj","ababababkk","ababababkl","ababababkm","ababababkn","ababababko","ababababkp","ababababkq","ababababkr","ababababks","ababababkt","ababababku","ababababkv","ababababkw","ababababkx","ababababky","ababababkz","ababababla","abababablb","abababablc","ababababld","abababable","abababablf","abababablg","abababablh","ababababli","abababablj","abababablk","ababababll","abababablm","ababababln","abababablo","abababablp","abababablq","abababablr","ababababls","abababablt","abababablu","abababablv","abababablw","abababablx","abababably","abababablz","ababababma","ababababmb","ababababmc","ababababmd","ababababme","ababababmf","ababababmg","ababababmh","ababababmi","ababababmj","ababababmk","ababababml","ababababmm","ababababmn","ababababmo","ababababmp","ababababmq","ababababmr","ababababms","ababababmt","ababababmu","ababababmv","ababababmw","ababababmx","ababababmy","ababababmz","ababababna","ababababnb","ababababnc","ababababnd","ababababne","ababababnf","ababababng","ababababnh","ababababni","ababababnj","ababababnk","ababababnl","ababababnm","ababababnn","ababababno","ababababnp","ababababnq","ababababnr","ababababns","ababababnt","ababababnu","ababababnv","ababababnw","ababababnx","ababababny","ababababnz","ababababoa","ababababob","ababababoc","ababababod","ababababoe","ababababof","ababababog","ababababoh","ababababoi","ababababoj","ababababok","ababababol","ababababom","ababababon","ababababoo","ababababop","ababababoq","ababababor","ababababos","ababababot","ababababou","ababababov","ababababow","ababababox","ababababoy","ababababoz","ababababpa","ababababpb","ababababpc","ababababpd","ababababpe","ababababpf","ababababpg","ababababph","ababababpi","ababababpj","ababababpk","ababababpl","ababababpm","ababababpn","ababababpo","ababababpp","ababababpq","ababababpr","ababababps","ababababpt","ababababpu","ababababpv","ababababpw","ababababpx","ababababpy","ababababpz","ababababqa","ababababqb","ababababqc","ababababqd","ababababqe","ababababqf","ababababqg","ababababqh","ababababqi","ababababqj","ababababqk","ababababql","ababababqm","ababababqn","ababababqo","ababababqp","ababababqq","ababababqr","ababababqs","ababababqt","ababababqu","ababababqv","ababababqw","ababababqx","ababababqy","ababababqz","ababababra","ababababrb","ababababrc","ababababrd","ababababre","ababababrf","ababababrg","ababababrh","ababababri","ababababrj","ababababrk","ababababrl","ababababrm","ababababrn","ababababro","ababababrp","ababababrq","ababababrr","ababababrs","ababababrt","ababababru","ababababrv","ababababrw","ababababrx","ababababry","ababababrz","ababababsa","ababababsb","ababababsc","ababababsd","ababababse","ababababsf","ababababsg","ababababsh","ababababsi","ababababsj","ababababsk","ababababsl","ababababsm","ababababsn","ababababso","ababababsp","ababababsq","ababababsr","ababababss","ababababst","ababababsu","ababababsv","ababababsw","ababababsx","ababababsy","ababababsz","ababababta","ababababtb","ababababtc","ababababtd","ababababte","ababababtf","ababababtg","ababababth","ababababti","ababababtj","ababababtk","ababababtl","ababababtm","ababababtn","ababababto","ababababtp","ababababtq","ababababtr","ababababts","ababababtt","ababababtu","ababababtv","ababababtw","ababababtx","ababababty","ababababtz","ababababua","ababababub","ababababuc","ababababud","ababababue","ababababuf","ababababug","ababababuh","ababababui","ababababuj","ababababuk","ababababul","ababababum","ababababun","ababababuo","ababababup","ababababuq","ababababur","ababababus","ababababut","ababababuu","ababababuv","ababababuw","ababababux","ababababuy","ababababuz","ababababva","ababababvb","ababababvc","ababababvd","ababababve","ababababvf","ababababvg","ababababvh","ababababvi","ababababvj","ababababvk","ababababvl","ababababvm","ababababvn","ababababvo","ababababvp","ababababvq","ababababvr","ababababvs","ababababvt","ababababvu","ababababvv","ababababvw","ababababvx","ababababvy","ababababvz","ababababwa","ababababwb","ababababwc","ababababwd","ababababwe","ababababwf","ababababwg","ababababwh","ababababwi","ababababwj","ababababwk","ababababwl","ababababwm","ababababwn","ababababwo","ababababwp","ababababwq","ababababwr","ababababws","ababababwt","ababababwu","ababababwv","ababababww","ababababwx","ababababwy","ababababwz","ababababxa","ababababxb","ababababxc","ababababxd","ababababxe","ababababxf","ababababxg","ababababxh","ababababxi","ababababxj","ababababxk","ababababxl","ababababxm","ababababxn","ababababxo","ababababxp","ababababxq","ababababxr","ababababxs","ababababxt","ababababxu","ababababxv","ababababxw","ababababxx","ababababxy","ababababxz","ababababya","ababababyb","ababababyc","ababababyd","ababababye","ababababyf","ababababyg","ababababyh","ababababyi","ababababyj","ababababyk","ababababyl","ababababym","ababababyn","ababababyo","ababababyp","ababababyq","ababababyr","ababababys","ababababyt","ababababyu","ababababyv","ababababyw","ababababyx","ababababyy","ababababyz","ababababza","ababababzb","ababababzc","ababababzd","ababababze","ababababzf","ababababzg","ababababzh","ababababzi","ababababzj","ababababzk","ababababzl","ababababzm","ababababzn","ababababzo","ababababzp","ababababzq","ababababzr","ababababzs","ababababzt","ababababzu","ababababzv","ababababzw","ababababzx","ababababzy","ababababzz"]
        # ],
        [
            [["a","b","e"],["b","c","d"]],
            ["abcdeb"]
        ],
    ]

    for board, words in board_words:
        print(Solution().findWords(board, words))

word: abcdeb match: (0, (1, 1))
word: abcdeb match: (1, (1, 2))
word: abcdeb match: (2, (2, 2))
word: abcdeb match: (3, (2, 3))
word: abcdeb match: (4, (1, 3))
word: abcdeb match: (1, (2, 1))
[]


好家伙好家伙，被测试用例克制的死死的。

好吧好吧，看来需要把路径作为cache的key。\
路径还不能只是简单的字母，\
还需要每一步的位置。（这个用例真行）

In [20]:
from typing import List
from functools import lru_cache
# from pprint import pprint


class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 0.哨
        board = [['', *b, ''] for b in board]
        board.insert(0, [''] * len(board[0]))
        board.append([''] * len(board[0]))
        # print(board)
        # 1.先将开头字母的索引记录下来
        start_letter = {}
        for x in words:
            start_letter[x[0]] = []
        # print(start_letter)
        for r in range(len(board)):
            for c in range(len(board[0])):
                char = board[r][c]
                if char in start_letter:
                    start_letter[char].append([r, c])
        # print(start_letter)
        # 2.搜索回溯
        @lru_cache()
        def dfs(word, match, path):
            # print('word:', word, 'match:', match, 'path:', path)
            # pprint(vis)
            # print('####\n\n')
            i, (r, c) = match

            if i == len(word)-1:
                return True

            stacks = []
            vis[r][c] = True
            direction = [(0, -1), (0, 1), (-1, 0), (1, 0)]
            for x, y in direction:
                if board[r+x][c+y] == word[i+1] and not vis[r+x][c+y]:   # 匹配字符，且未访问
                    stacks.append((i+1, (r+x, c+y)))
            result = []
            for i, (r, c) in stacks:                     # 用for循环代替stack，才能回滚访问记录 到分叉点
                # result += [dfs(word, (i, (r, c)), path+word[i])]                  # 将每一步的字母位置也作为key，不超时，但显然是还有哪个问题的
                result += [dfs(word, (i, (r, c)), path+f'-{word[i]}{i}.{r}.{c}')]   # 将每一步的字母位置也作为key,🍀，又超时
                vis[r][c] = False
            return any(result)

        ans = []
        for word in words:
            if start_letter[word[0]] == []:
                continue
            if len(word) == 1:
                ans.append(word)
                continue
            for start_pos in start_letter[word[0]]:
                match = (0, tuple(start_pos))  # board的起始坐标，word的匹配字符索引
                vis = [[False] * len(board[0]) for _ in range(len(board))]
                if dfs(word, match, word[0]):
                    ans.append(word)
                    break           # 如果在一个地方匹配成功，就不再继续
            # print('----')
        return ans



if __name__ == '__main__':
    board_words = [
        [
            [["o","a","t","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
            ["oath","pea","eat","rain"],
        ],
        [
            [["a","b"],["c","d"]],
            ["abcb"],
        ],
        [
            [["a"]],
            ["a"],
        ],
        [
            [["a", "b"], ["c", "d"]],
            ["ac"],
        ],
        [
            [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]],
            ["oa","oaa"],
        ],
        [
            [["a","f","c"],["a","e","d"],["a","f","g"]],
            ["eaafcdgfa", "eaf"]
        ],
        [
            [["a"]],
            ["ab"]
        ],
        [
            [["a","b"],["a","a"]],
            ["bab", "aaba"]
        ],
        [
            [["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"]],
            ["ababababaa","ababababab","ababababac","ababababad","ababababae","ababababaf","ababababag","ababababah","ababababai","ababababaj","ababababak","ababababal","ababababam","ababababan","ababababao","ababababap","ababababaq","ababababar","ababababas","ababababat","ababababau","ababababav","ababababaw","ababababax","ababababay","ababababaz","ababababba","ababababbb","ababababbc","ababababbd","ababababbe","ababababbf","ababababbg","ababababbh","ababababbi","ababababbj","ababababbk","ababababbl","ababababbm","ababababbn","ababababbo","ababababbp","ababababbq","ababababbr","ababababbs","ababababbt","ababababbu","ababababbv","ababababbw","ababababbx","ababababby","ababababbz","ababababca","ababababcb","ababababcc","ababababcd","ababababce","ababababcf","ababababcg","ababababch","ababababci","ababababcj","ababababck","ababababcl","ababababcm","ababababcn","ababababco","ababababcp","ababababcq","ababababcr","ababababcs","ababababct","ababababcu","ababababcv","ababababcw","ababababcx","ababababcy","ababababcz","ababababda","ababababdb","ababababdc","ababababdd","ababababde","ababababdf","ababababdg","ababababdh","ababababdi","ababababdj","ababababdk","ababababdl","ababababdm","ababababdn","ababababdo","ababababdp","ababababdq","ababababdr","ababababds","ababababdt","ababababdu","ababababdv","ababababdw","ababababdx","ababababdy","ababababdz","ababababea","ababababeb","ababababec","ababababed","ababababee","ababababef","ababababeg","ababababeh","ababababei","ababababej","ababababek","ababababel","ababababem","ababababen","ababababeo","ababababep","ababababeq","ababababer","ababababes","ababababet","ababababeu","ababababev","ababababew","ababababex","ababababey","ababababez","ababababfa","ababababfb","ababababfc","ababababfd","ababababfe","ababababff","ababababfg","ababababfh","ababababfi","ababababfj","ababababfk","ababababfl","ababababfm","ababababfn","ababababfo","ababababfp","ababababfq","ababababfr","ababababfs","ababababft","ababababfu","ababababfv","ababababfw","ababababfx","ababababfy","ababababfz","ababababga","ababababgb","ababababgc","ababababgd","ababababge","ababababgf","ababababgg","ababababgh","ababababgi","ababababgj","ababababgk","ababababgl","ababababgm","ababababgn","ababababgo","ababababgp","ababababgq","ababababgr","ababababgs","ababababgt","ababababgu","ababababgv","ababababgw","ababababgx","ababababgy","ababababgz","ababababha","ababababhb","ababababhc","ababababhd","ababababhe","ababababhf","ababababhg","ababababhh","ababababhi","ababababhj","ababababhk","ababababhl","ababababhm","ababababhn","ababababho","ababababhp","ababababhq","ababababhr","ababababhs","ababababht","ababababhu","ababababhv","ababababhw","ababababhx","ababababhy","ababababhz","ababababia","ababababib","ababababic","ababababid","ababababie","ababababif","ababababig","ababababih","ababababii","ababababij","ababababik","ababababil","ababababim","ababababin","ababababio","ababababip","ababababiq","ababababir","ababababis","ababababit","ababababiu","ababababiv","ababababiw","ababababix","ababababiy","ababababiz","ababababja","ababababjb","ababababjc","ababababjd","ababababje","ababababjf","ababababjg","ababababjh","ababababji","ababababjj","ababababjk","ababababjl","ababababjm","ababababjn","ababababjo","ababababjp","ababababjq","ababababjr","ababababjs","ababababjt","ababababju","ababababjv","ababababjw","ababababjx","ababababjy","ababababjz","ababababka","ababababkb","ababababkc","ababababkd","ababababke","ababababkf","ababababkg","ababababkh","ababababki","ababababkj","ababababkk","ababababkl","ababababkm","ababababkn","ababababko","ababababkp","ababababkq","ababababkr","ababababks","ababababkt","ababababku","ababababkv","ababababkw","ababababkx","ababababky","ababababkz","ababababla","abababablb","abababablc","ababababld","abababable","abababablf","abababablg","abababablh","ababababli","abababablj","abababablk","ababababll","abababablm","ababababln","abababablo","abababablp","abababablq","abababablr","ababababls","abababablt","abababablu","abababablv","abababablw","abababablx","abababably","abababablz","ababababma","ababababmb","ababababmc","ababababmd","ababababme","ababababmf","ababababmg","ababababmh","ababababmi","ababababmj","ababababmk","ababababml","ababababmm","ababababmn","ababababmo","ababababmp","ababababmq","ababababmr","ababababms","ababababmt","ababababmu","ababababmv","ababababmw","ababababmx","ababababmy","ababababmz","ababababna","ababababnb","ababababnc","ababababnd","ababababne","ababababnf","ababababng","ababababnh","ababababni","ababababnj","ababababnk","ababababnl","ababababnm","ababababnn","ababababno","ababababnp","ababababnq","ababababnr","ababababns","ababababnt","ababababnu","ababababnv","ababababnw","ababababnx","ababababny","ababababnz","ababababoa","ababababob","ababababoc","ababababod","ababababoe","ababababof","ababababog","ababababoh","ababababoi","ababababoj","ababababok","ababababol","ababababom","ababababon","ababababoo","ababababop","ababababoq","ababababor","ababababos","ababababot","ababababou","ababababov","ababababow","ababababox","ababababoy","ababababoz","ababababpa","ababababpb","ababababpc","ababababpd","ababababpe","ababababpf","ababababpg","ababababph","ababababpi","ababababpj","ababababpk","ababababpl","ababababpm","ababababpn","ababababpo","ababababpp","ababababpq","ababababpr","ababababps","ababababpt","ababababpu","ababababpv","ababababpw","ababababpx","ababababpy","ababababpz","ababababqa","ababababqb","ababababqc","ababababqd","ababababqe","ababababqf","ababababqg","ababababqh","ababababqi","ababababqj","ababababqk","ababababql","ababababqm","ababababqn","ababababqo","ababababqp","ababababqq","ababababqr","ababababqs","ababababqt","ababababqu","ababababqv","ababababqw","ababababqx","ababababqy","ababababqz","ababababra","ababababrb","ababababrc","ababababrd","ababababre","ababababrf","ababababrg","ababababrh","ababababri","ababababrj","ababababrk","ababababrl","ababababrm","ababababrn","ababababro","ababababrp","ababababrq","ababababrr","ababababrs","ababababrt","ababababru","ababababrv","ababababrw","ababababrx","ababababry","ababababrz","ababababsa","ababababsb","ababababsc","ababababsd","ababababse","ababababsf","ababababsg","ababababsh","ababababsi","ababababsj","ababababsk","ababababsl","ababababsm","ababababsn","ababababso","ababababsp","ababababsq","ababababsr","ababababss","ababababst","ababababsu","ababababsv","ababababsw","ababababsx","ababababsy","ababababsz","ababababta","ababababtb","ababababtc","ababababtd","ababababte","ababababtf","ababababtg","ababababth","ababababti","ababababtj","ababababtk","ababababtl","ababababtm","ababababtn","ababababto","ababababtp","ababababtq","ababababtr","ababababts","ababababtt","ababababtu","ababababtv","ababababtw","ababababtx","ababababty","ababababtz","ababababua","ababababub","ababababuc","ababababud","ababababue","ababababuf","ababababug","ababababuh","ababababui","ababababuj","ababababuk","ababababul","ababababum","ababababun","ababababuo","ababababup","ababababuq","ababababur","ababababus","ababababut","ababababuu","ababababuv","ababababuw","ababababux","ababababuy","ababababuz","ababababva","ababababvb","ababababvc","ababababvd","ababababve","ababababvf","ababababvg","ababababvh","ababababvi","ababababvj","ababababvk","ababababvl","ababababvm","ababababvn","ababababvo","ababababvp","ababababvq","ababababvr","ababababvs","ababababvt","ababababvu","ababababvv","ababababvw","ababababvx","ababababvy","ababababvz","ababababwa","ababababwb","ababababwc","ababababwd","ababababwe","ababababwf","ababababwg","ababababwh","ababababwi","ababababwj","ababababwk","ababababwl","ababababwm","ababababwn","ababababwo","ababababwp","ababababwq","ababababwr","ababababws","ababababwt","ababababwu","ababababwv","ababababww","ababababwx","ababababwy","ababababwz","ababababxa","ababababxb","ababababxc","ababababxd","ababababxe","ababababxf","ababababxg","ababababxh","ababababxi","ababababxj","ababababxk","ababababxl","ababababxm","ababababxn","ababababxo","ababababxp","ababababxq","ababababxr","ababababxs","ababababxt","ababababxu","ababababxv","ababababxw","ababababxx","ababababxy","ababababxz","ababababya","ababababyb","ababababyc","ababababyd","ababababye","ababababyf","ababababyg","ababababyh","ababababyi","ababababyj","ababababyk","ababababyl","ababababym","ababababyn","ababababyo","ababababyp","ababababyq","ababababyr","ababababys","ababababyt","ababababyu","ababababyv","ababababyw","ababababyx","ababababyy","ababababyz","ababababza","ababababzb","ababababzc","ababababzd","ababababze","ababababzf","ababababzg","ababababzh","ababababzi","ababababzj","ababababzk","ababababzl","ababababzm","ababababzn","ababababzo","ababababzp","ababababzq","ababababzr","ababababzs","ababababzt","ababababzu","ababababzv","ababababzw","ababababzx","ababababzy","ababababzz"]
        ],
        [
            [["a","b","e"],["b","c","d"]],
            ["abcdeb"]
        ],
    ]

    for board, words in board_words:
        print(Solution().findWords(board, words))

['oath', 'eat']
[]
['a']
['ac']
['oa', 'oaa']
['eaafcdgfa']
[]
['aaba']
['ababababab']
['abcdeb']


# 4

无情折磨，好吧好吧，观察超时的用例，貌似先过滤一波字符就行了。

> （这困难题，没有思想上的难度，纯粹像工程上的问题。）

貌似缓存也可以不要了

In [29]:
from typing import List
# from pprint import pprint


class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 0.哨
        board = [['', *b, ''] for b in board]
        board.insert(0, [''] * len(board[0]))
        board.append([''] * len(board[0]))
        # print(board)
        # 1.先将开头字母的索引记录下来
        start_letter = {}
        for x in words:
            start_letter[x[0]] = []
        # print(start_letter)
        for r in range(len(board)):
            for c in range(len(board[0])):
                char = board[r][c]
                if char in start_letter:
                    start_letter[char].append([r, c])
        # print(start_letter)
        # 2.搜索回溯
        def dfs(word, match):
            # print('word:', word, 'match:', match, 'path:', path)
            # pprint(vis)
            # print('####\n\n')
            i, (r, c) = match

            if i == len(word)-1:
                return True

            stacks = []
            vis[r][c] = True
            direction = [(0, -1), (0, 1), (-1, 0), (1, 0)]
            for x, y in direction:
                if board[r+x][c+y] == word[i+1] and not vis[r+x][c+y]:   # 匹配字符，且未访问
                    stacks.append([i+1, (r+x, c+y)])
            result = []
            for i, (r, c) in stacks:                     # 用for循环代替stack，才能回滚访问记录 到分叉点
                result += [dfs(word, [i, (r, c)])]
                vis[r][c] = False
            return any(result)
        
        def exist_letter(board_set, word):          # 判断字母集合是否匹配
            word_set = set(list(word))
            return word_set & board_set == word_set

        board_set = []
        [board_set.extend(x) for x in board]
        board_set = set(board_set)
        ans = []
        for word in words:
            if not exist_letter(board_set, word):                # 如果字母集合不能匹配哪么，直接跳过
                continue
            if start_letter[word[0]] == []:
                continue
            if len(word) == 1:
                ans.append(word)
                continue
            for start_pos in start_letter[word[0]]:
                match = (0, start_pos)  # board的起始坐标，word的匹配字符索引
                vis = [[False] * len(board[0]) for _ in range(len(board))]
                if dfs(word, match):
                    ans.append(word)
                    break           # 如果在一个地方匹配成功，就不再继续
            # print('----')
        return ans


if __name__ == '__main__':
    board_words = [
        [
            [["o","a","t","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]],
            ["oath","pea","eat","rain"],
        ],
        [
            [["a","b"],["c","d"]],
            ["abcb"],
        ],
        [
            [["a"]],
            ["a"],
        ],
        [
            [["a", "b"], ["c", "d"]],
            ["ac"],
        ],
        [
            [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]],
            ["oa","oaa"],
        ],
        [
            [["a","f","c"],["a","e","d"],["a","f","g"]],
            ["eaafcdgfa", "eaf"]
        ],
        [
            [["a"]],
            ["ab"]
        ],
        [
            [["a","b"],["a","a"]],
            ["bab", "aaba"]
        ],
        [
            [["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"],["b","a","b","a","b","a","b","a","b","a"],["a","b","a","b","a","b","a","b","a","b"]],
            ["ababababaa","ababababab","ababababac","ababababad","ababababae","ababababaf","ababababag","ababababah","ababababai","ababababaj","ababababak","ababababal","ababababam","ababababan","ababababao","ababababap","ababababaq","ababababar","ababababas","ababababat","ababababau","ababababav","ababababaw","ababababax","ababababay","ababababaz","ababababba","ababababbb","ababababbc","ababababbd","ababababbe","ababababbf","ababababbg","ababababbh","ababababbi","ababababbj","ababababbk","ababababbl","ababababbm","ababababbn","ababababbo","ababababbp","ababababbq","ababababbr","ababababbs","ababababbt","ababababbu","ababababbv","ababababbw","ababababbx","ababababby","ababababbz","ababababca","ababababcb","ababababcc","ababababcd","ababababce","ababababcf","ababababcg","ababababch","ababababci","ababababcj","ababababck","ababababcl","ababababcm","ababababcn","ababababco","ababababcp","ababababcq","ababababcr","ababababcs","ababababct","ababababcu","ababababcv","ababababcw","ababababcx","ababababcy","ababababcz","ababababda","ababababdb","ababababdc","ababababdd","ababababde","ababababdf","ababababdg","ababababdh","ababababdi","ababababdj","ababababdk","ababababdl","ababababdm","ababababdn","ababababdo","ababababdp","ababababdq","ababababdr","ababababds","ababababdt","ababababdu","ababababdv","ababababdw","ababababdx","ababababdy","ababababdz","ababababea","ababababeb","ababababec","ababababed","ababababee","ababababef","ababababeg","ababababeh","ababababei","ababababej","ababababek","ababababel","ababababem","ababababen","ababababeo","ababababep","ababababeq","ababababer","ababababes","ababababet","ababababeu","ababababev","ababababew","ababababex","ababababey","ababababez","ababababfa","ababababfb","ababababfc","ababababfd","ababababfe","ababababff","ababababfg","ababababfh","ababababfi","ababababfj","ababababfk","ababababfl","ababababfm","ababababfn","ababababfo","ababababfp","ababababfq","ababababfr","ababababfs","ababababft","ababababfu","ababababfv","ababababfw","ababababfx","ababababfy","ababababfz","ababababga","ababababgb","ababababgc","ababababgd","ababababge","ababababgf","ababababgg","ababababgh","ababababgi","ababababgj","ababababgk","ababababgl","ababababgm","ababababgn","ababababgo","ababababgp","ababababgq","ababababgr","ababababgs","ababababgt","ababababgu","ababababgv","ababababgw","ababababgx","ababababgy","ababababgz","ababababha","ababababhb","ababababhc","ababababhd","ababababhe","ababababhf","ababababhg","ababababhh","ababababhi","ababababhj","ababababhk","ababababhl","ababababhm","ababababhn","ababababho","ababababhp","ababababhq","ababababhr","ababababhs","ababababht","ababababhu","ababababhv","ababababhw","ababababhx","ababababhy","ababababhz","ababababia","ababababib","ababababic","ababababid","ababababie","ababababif","ababababig","ababababih","ababababii","ababababij","ababababik","ababababil","ababababim","ababababin","ababababio","ababababip","ababababiq","ababababir","ababababis","ababababit","ababababiu","ababababiv","ababababiw","ababababix","ababababiy","ababababiz","ababababja","ababababjb","ababababjc","ababababjd","ababababje","ababababjf","ababababjg","ababababjh","ababababji","ababababjj","ababababjk","ababababjl","ababababjm","ababababjn","ababababjo","ababababjp","ababababjq","ababababjr","ababababjs","ababababjt","ababababju","ababababjv","ababababjw","ababababjx","ababababjy","ababababjz","ababababka","ababababkb","ababababkc","ababababkd","ababababke","ababababkf","ababababkg","ababababkh","ababababki","ababababkj","ababababkk","ababababkl","ababababkm","ababababkn","ababababko","ababababkp","ababababkq","ababababkr","ababababks","ababababkt","ababababku","ababababkv","ababababkw","ababababkx","ababababky","ababababkz","ababababla","abababablb","abababablc","ababababld","abababable","abababablf","abababablg","abababablh","ababababli","abababablj","abababablk","ababababll","abababablm","ababababln","abababablo","abababablp","abababablq","abababablr","ababababls","abababablt","abababablu","abababablv","abababablw","abababablx","abababably","abababablz","ababababma","ababababmb","ababababmc","ababababmd","ababababme","ababababmf","ababababmg","ababababmh","ababababmi","ababababmj","ababababmk","ababababml","ababababmm","ababababmn","ababababmo","ababababmp","ababababmq","ababababmr","ababababms","ababababmt","ababababmu","ababababmv","ababababmw","ababababmx","ababababmy","ababababmz","ababababna","ababababnb","ababababnc","ababababnd","ababababne","ababababnf","ababababng","ababababnh","ababababni","ababababnj","ababababnk","ababababnl","ababababnm","ababababnn","ababababno","ababababnp","ababababnq","ababababnr","ababababns","ababababnt","ababababnu","ababababnv","ababababnw","ababababnx","ababababny","ababababnz","ababababoa","ababababob","ababababoc","ababababod","ababababoe","ababababof","ababababog","ababababoh","ababababoi","ababababoj","ababababok","ababababol","ababababom","ababababon","ababababoo","ababababop","ababababoq","ababababor","ababababos","ababababot","ababababou","ababababov","ababababow","ababababox","ababababoy","ababababoz","ababababpa","ababababpb","ababababpc","ababababpd","ababababpe","ababababpf","ababababpg","ababababph","ababababpi","ababababpj","ababababpk","ababababpl","ababababpm","ababababpn","ababababpo","ababababpp","ababababpq","ababababpr","ababababps","ababababpt","ababababpu","ababababpv","ababababpw","ababababpx","ababababpy","ababababpz","ababababqa","ababababqb","ababababqc","ababababqd","ababababqe","ababababqf","ababababqg","ababababqh","ababababqi","ababababqj","ababababqk","ababababql","ababababqm","ababababqn","ababababqo","ababababqp","ababababqq","ababababqr","ababababqs","ababababqt","ababababqu","ababababqv","ababababqw","ababababqx","ababababqy","ababababqz","ababababra","ababababrb","ababababrc","ababababrd","ababababre","ababababrf","ababababrg","ababababrh","ababababri","ababababrj","ababababrk","ababababrl","ababababrm","ababababrn","ababababro","ababababrp","ababababrq","ababababrr","ababababrs","ababababrt","ababababru","ababababrv","ababababrw","ababababrx","ababababry","ababababrz","ababababsa","ababababsb","ababababsc","ababababsd","ababababse","ababababsf","ababababsg","ababababsh","ababababsi","ababababsj","ababababsk","ababababsl","ababababsm","ababababsn","ababababso","ababababsp","ababababsq","ababababsr","ababababss","ababababst","ababababsu","ababababsv","ababababsw","ababababsx","ababababsy","ababababsz","ababababta","ababababtb","ababababtc","ababababtd","ababababte","ababababtf","ababababtg","ababababth","ababababti","ababababtj","ababababtk","ababababtl","ababababtm","ababababtn","ababababto","ababababtp","ababababtq","ababababtr","ababababts","ababababtt","ababababtu","ababababtv","ababababtw","ababababtx","ababababty","ababababtz","ababababua","ababababub","ababababuc","ababababud","ababababue","ababababuf","ababababug","ababababuh","ababababui","ababababuj","ababababuk","ababababul","ababababum","ababababun","ababababuo","ababababup","ababababuq","ababababur","ababababus","ababababut","ababababuu","ababababuv","ababababuw","ababababux","ababababuy","ababababuz","ababababva","ababababvb","ababababvc","ababababvd","ababababve","ababababvf","ababababvg","ababababvh","ababababvi","ababababvj","ababababvk","ababababvl","ababababvm","ababababvn","ababababvo","ababababvp","ababababvq","ababababvr","ababababvs","ababababvt","ababababvu","ababababvv","ababababvw","ababababvx","ababababvy","ababababvz","ababababwa","ababababwb","ababababwc","ababababwd","ababababwe","ababababwf","ababababwg","ababababwh","ababababwi","ababababwj","ababababwk","ababababwl","ababababwm","ababababwn","ababababwo","ababababwp","ababababwq","ababababwr","ababababws","ababababwt","ababababwu","ababababwv","ababababww","ababababwx","ababababwy","ababababwz","ababababxa","ababababxb","ababababxc","ababababxd","ababababxe","ababababxf","ababababxg","ababababxh","ababababxi","ababababxj","ababababxk","ababababxl","ababababxm","ababababxn","ababababxo","ababababxp","ababababxq","ababababxr","ababababxs","ababababxt","ababababxu","ababababxv","ababababxw","ababababxx","ababababxy","ababababxz","ababababya","ababababyb","ababababyc","ababababyd","ababababye","ababababyf","ababababyg","ababababyh","ababababyi","ababababyj","ababababyk","ababababyl","ababababym","ababababyn","ababababyo","ababababyp","ababababyq","ababababyr","ababababys","ababababyt","ababababyu","ababababyv","ababababyw","ababababyx","ababababyy","ababababyz","ababababza","ababababzb","ababababzc","ababababzd","ababababze","ababababzf","ababababzg","ababababzh","ababababzi","ababababzj","ababababzk","ababababzl","ababababzm","ababababzn","ababababzo","ababababzp","ababababzq","ababababzr","ababababzs","ababababzt","ababababzu","ababababzv","ababababzw","ababababzx","ababababzy","ababababzz"]
        ],
        [
            [["a","b","e"],["b","c","d"]],
            ["abcdeb"]
        ],
    ]

    for board, words in board_words:
        print(Solution().findWords(board, words))

['oath', 'eat']
[]
['a']
['ac']
['oa', 'oaa']
['eaafcdgfa']
[]
['aaba']
['ababababab']
['abcdeb']


43/43 cases passed (504 ms)\
Your runtime beats 71.94 % of python3 submissions\
Your memory usage beats 64.98 % of python3 submissions (15.3 MB)

time: O(n^2)

space: O(n)

> n是字符网格的数量，路径可能存在假路，可能会回溯。

# 题解

尽管做出来了，但官方题解的`前缀树`方法的确更加简洁。

> 利用了208题的方法。

> 不需要额外处理exist_letter，明显不如官方。